In [5]:
import sys
from timeit import default_timer as timer

sys.path.append('../pmm/')

import inputs
import pmm
import conversions
import MDAnalysis as mda
import numpy as np
#import numpy.linalg 
import nglview as nv
from nglview.datafiles import GRO, XTC

In [2]:
# gather the electronic properties
start = timer()
pmm_inputs = inputs.get_tot_input_gauss('thioindigo/thio_pbe0_esp{}_x180.log', 16)
end = timer()
print('inputs da Gaussian ci mette: ', end - start)
#print(pmm_inputs)

qc_geom = pmm.convert2Universe(pmm_inputs['geometry'])
print(qc_geom)


inputs da Gaussian ci mette:  0.02704309999535326
<Universe with 28 atoms>
/mnt/d/miniconda3/envs/pmm/lib/python3.9/site-packages/MDAnalysis/core/universe.py:467: UserWarning: Residues specified but no atom_resindex given.  All atoms will be placed in first Residue.
  warnings.warn(
/mnt/d/miniconda3/envs/pmm/lib/python3.9/site-packages/MDAnalysis/core/universe.py:473: UserWarning: Segments specified but no segment_resindex given.  All residues will be placed in first Segment
  warnings.warn(


In [3]:
strat = timer()
traj = mda.Universe('thioindigo/thioindigo_trans_etoh_md.tpr', 'thioindigo/thio_etoh_center_10ps.xtc')
end = timer()
print('leggere xtc ci mette: ', end - start)
#w = nv.show_mdanalysis(traj)
#w

#qc_traj, solv_traj = pmm.split_qc_solv(traj, '1:28')
#print(qc_traj)
#w = nv.show_mdanalysis(solv_traj)
#w

leggere xtc ci mette:  5.43941829999676


In [13]:
start = timer()
pmm_traj = []
for ts in traj.trajectory:
    qc_traj, solv_traj = pmm.split_qc_solv(traj, '1:28')
    #print(pmm.rotate_dip_matrix(pmm_inputs['dip_matrix'], qc_traj, qc_geom))
    cdm = qc_traj.center_of_mass()
    #print(solv_traj.atoms.positions)
    el_field, potential = pmm.calc_el_field_pot(solv_traj.atoms.positions, solv_traj.atoms.charges, cdm)
    #print(el_field, potential)
    rot_dip_matrix = pmm.rotate_dip_matrix(pmm_inputs['dip_matrix'], qc_traj, qc_geom)
    pmm_matrix = pmm.pmm_matrix(pmm_inputs['energies'], rot_dip_matrix, el_field, potential)
    #print(pmm_matrix)
    pmm_traj.append(np.linalg.eig(pmm_matrix))
end = timer()
print('PMM ci mette: ', end - start)
print(len(pmm_traj), pmm_traj[0][0].shape)


PMM ci mette:  0.06523870000091847
11 (16,)


In [11]:
matrix = np.diag(pmm_inputs['energies'])
matrix

array([[-1560.41534804,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ],
       [    0.        , -1560.32645881,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ],
       [    0.        ,     0.        , -1560.31062353,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ,
            0.        ,     0.        ,     0.        ,     0.        ],
       [    0.        ,     0.        ,     0.        , -1560.30687143,
            0.        ,     0.        ,     0.        ,     0